# JetRacer車両制御の基礎
ここではJetRacerのモーターとサーボをプログラム的に制御する基礎的な部分を学びます。  
Jupyterノートブックではプログラムをブロック毎に実行します。ここに書いてあるプログラムは実際に車体を動かすために使われているプログラム総量のほんのわずかですが、車体制御に関する重要なパラメータ部分を扱います。  

それでは実際に実行してみましょう。
まずは[NvidiaRacecar](https://github.com/FaBoPlatform/jetracer/blob/master/jetracer/nvidia_racecar.py)クラスをインスタンス化しましょう。

In [ ]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

次に、車体固有のパラメータを設定します。  
RC Carは通常、HIGHになる電圧時間が **1000μsから2000μs** の範囲となる信号波を **66.6Hz** の周期で流す **PWM信号** で制御されています。  
実際はメーカーや機器によって様々な周期で動作しています。(50Hzから400Hz超えまで)  
* ステアリング操作はPWM信号をサーボに流すことでおこないます。
* スロットル操作はPWM信号をESC(エレクトリック・スピード・コントローラ)に流すことでおこないます。  
PWM制御できるRC Carは、プログラム制御する際にはほぼ例外なくPCA9685が使われています。JetRacerでもAdafruitのライブラリを使ってPCA9685を利用しています。そのため、**ここでは直接PWMを扱うコードは出てきません**が、RC Carを制御するためのいくつかの知識が必要になります。  

タミヤTT-02 XBに搭載されているメカ（サーボ、ESC）の場合、制御範囲のPWM値はおおむね以下の値になります。
* ESCのニュートラルは1540μs (信号電圧がHIGHになっている時間)
* 前進方向：1480 - 1100μs (プロポの信号)
* 後進方向：1600 - 1960μs (プロポの信号)
* ニュートラル範囲：1480 - 1600μs
* サーボのニュートラルは1540μs
* 左方向：1540 - 1100μs (プロポの信号。実際のところ、物理的に動作可能な範囲は1250μs程度まで)
* 右方向：1540 - 1960μs (プロポの信号。実際のところ、物理的に動作可能な範囲は1750μs程度まで)
* 周期：66.67Hz

これらの値は送信機のトリム調整(ニュートラル調整)で変わってきます。受信機のGNDと信号線をオシロスコープに接続して観察することができます。  
車体や組み立て方の違いや、メーカー、メカ、設定の違いによっては前後左右が逆の場合や、ニュートラルやエンドポイントが異なることがあります。  

そして**最も重要な点**ですが、JetRacerは**adafruit-circuitpython-servokit-1.2.2**ライブラリを使ってPCA9685を制御することでPWM信号を生成しています。このライブラリは設定値[-1.0, 1.0]の範囲において、**[760, 2280]μs**、**50Hz**のPWM信号を発信します。これはRC CarのPWM範囲としては多すぎるため、サーボやモーターの動作として適切な範囲に制限する必要があります。  

### エンドポイント設定
RC Carのミドルクラス以上の送信機では標準的な機能となる**エンドポイント**と同じ機能を設定します。  
物理的にそれ以上左右に切れない状態でさらにステアリングを動作させようとするとサーボが壊れてしまうため、これ以上動作させないためのエンドポイントの設定はとても重要になります。  
エンドポイントではサーボとモーターの動作の限界点を設定します。  

|パラメータ|機能|値範囲|解説|
|:--|:--|:--|:--|
|car.steering_min_endpoint|左切れ角のエンドポイント|[-1.0, 1.0]|TT-02の場合は**-0.3**付近がちょうどいい値です。**steering_gain=1.0**、**steering_offset=0.0**、**steering=-1.0**の時にフロントタイヤが左いっぱいに切れている状態で、サーボからジリジリ音がしない値を設定します。0.01くらいの小さい範囲で調整します。|
|car.steering_max_endpoint|右切れ角のエンドポイント|[-1.0, 1.0]|TT-02の場合は**0.3**付近がちょうどいい値です。**steering_gain=1.0**、**steering_offset=0.0**、**steering=1.0**の時にフロントタイヤが右いっぱいに切れている状態で、サーボからジリジリ音がしない値を設定します。0.01くらいの小さい範囲で調整します。|
|car.throttle_min_endpoint|後進のエンドポイント|[-1.0, 1.0]|TT-02の場合は**-0.69**付近がちょうどいい値です。**throttle_gain=1.0**、**throttle_offset=0.0**、**throttle=-1.0**の時にモーターが最大速度で後進する値を設定します。|
|car.throttle_max_endpoint|前進のエンドポイント|[-1.0, 1.0]|TT-02の場合は**0.69**付近がちょうどいい値です。**throttle_gain=1.0**、**throttle_offset=0.0**、**throttle=1.0**の時にモーターが最大速度で前進する値を設定します。|


### ゲイン設定
ゲインではサーボとモーターの値に適用率を設定します。  
車両の基本操作ではステアリングとスロットルは共に**1.0**としておくことで車両は基本性能を発揮できますが、最初はスロットルゲインを**0.3**程度に設定しておいたほうが安全です。　　

|パラメータ|機能|値範囲|解説|
|:--|:--|:--|:--|
|car.steering_gain|ステアリング適用率|[-1.0, 1.0]|TT-02の場合は**1.0**にしておきます。**car.steering_gain**の値がプラスかマイナスかは、車種毎に固定になります（サーボの取り付け向きで決まります）。**car.steering**がプラスの時に右、マイナスの時に左にステアリングが切れるように**car.steering_gain**のプラスマイナスを決めます。|
|car.throttle_gain|スロットル適用率|[-1.0, 1.0]|TT-02の場合は最初は**-0.3**にしておきます。速度に慣れてきたら**-1.0**まで上げることができます。プラスかマイナスかはESCの仕様で決まります。**car.throttle**がプラスの時に前進するように**car.throttle_gain**のプラスマイナスを決めます。|


### 初期値とオフセット
ステアリングとスロットルの初期値とオフセットを設定します。  

|パラメータ|機能|値範囲|解説|
|:--|:--|:--|:--|
|car.steering|左右ステアリング値|[-1.0, 1.0]|現在のステアリングの値。0.0がニュートラル位置(理論上まっすぐ進む状態。実際は車体のがたつき、ゆがみ等でまっすぐ進まないことが多いです)。|
|car.steering_offset|ステアリングニュートラル補正値|[-1.0, 1.0]|車体がまっすぐ走行する位置に設定します。TT-02ノーマル車体の場合はステアリングのがたつきが大きく、完全にまっすぐ走行させることは不可能ですので、だいたいまっすぐ走行できればOKです。|
|car.throttle|前後スロットル値|[-1.0, 1.0]|現在のスロットルの値。0.0がニュートラル位置。|
|car.throttle_offset|スロットルニュートラル補正値|[-1.0, 1.0]|何もしないときに車体が停止する値に設定します。|

In [ ]:
# 車両パラメータを初期化します
car.steering_min_endpoint = -0.3 # 左切れ角のエンドポイント
car.steering_max_endpoint = 0.3 # 右切れ角のエンドポイント
car.throttle_min_endpoint = -0.69 # 後進のエンドポイント
car.throttle_max_endpoint = 0.69 # 前進のエンドポイント

car.steering = 0
car.steering_gain = 1.0
car.steering_offset = 0
car.throttle = 0
car.throttle_gain = -0.3
car.throttle_offset = 0

# 車両制御
それでは実際に車両を制御するためのスライドバーと入力ボックスを表示して、車両を動かしてみましょう。  
車両が動くので、周囲の安全を確保してから操作してください。　　
* **throttle**スライダー：上下にスライドするとタイヤがまわります。少しずつスライドしてください。この時、タイヤが後転する場合は**throttle_gain**のプラスマイナスを反対にしてください。後転させるためには、いちどニュートラル状態にしてから下にスライドする、ダブルアクション操作が必要になります。前進中に急にスライドを下にさげると、後転ではなくブレーキになります。
* **steering**スライダー：左右にスライドさせると、ステアリングが左右に動作します。左右逆に動作する場合は**steering_gain**のプラスマイナスを反対にしてください。

**steering_min_endpoint**、**steering_max_endpoint**、**throttle_min_endpoint**、**throttle_max_endpoint**、**steering_gain**、**steering_offset**、**throttle_gain**、**throttle_offset**の適切な値をみつけて、メモに残しておいてください。これらの値は自動走行時の車両パラメータにも設定することになります。

In [ ]:
import ipywidgets.widgets as widgets
from IPython.display import display
import traitlets

# create two sliders with range [-1.0, 1.0]
style = {'description_width': 'initial'}
steering_slider = widgets.FloatSlider(description='steering', style=style, min=-1.0, max=1.0, step=0.01, orientation='horizontal')
steering_gain = widgets.BoundedFloatText(description='steering_gain', style=style ,min=-1.0, max=1.0, step=0.01, value=car.steering_gain)
steering_offset = widgets.BoundedFloatText(description='steering_offset', style=style, min=-1.0, max=1.0, step=0.01, value=car.steering_offset)
throttle_slider = widgets.FloatSlider(description='throttle', style=style, min=-1.0, max=1.0, step=0.01, orientation='vertical')
throttle_gain = widgets.BoundedFloatText(description='throttle_gain', style=style, min=-1.0, max=1.0, step=0.01, value=car.throttle_gain)
throttle_offset = widgets.BoundedFloatText(description='throttle_offset', style=style, min=-1.0, max=1.0, step=0.01, value=car.throttle_offset)

# create a horizontal box container to place the sliders next to eachother
slider_container = widgets.HBox([throttle_slider, steering_slider])
slider_container.layout.align_items='center'
value_container = widgets.VBox([steering_gain, steering_offset, throttle_gain, throttle_offset])

control_container = widgets.HBox([slider_container, value_container])
control_container.layout.align_items='center'

# display the container in this cell's output
display(control_container)

# links
steering_link = traitlets.link((steering_slider, 'value'), (car, 'steering'))
steering_gain_link = traitlets.link((steering_gain, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_offset, 'value'), (car, 'steering_offset'))
throttle_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))
throttle_gain_link = traitlets.link((throttle_gain, 'value'), (car, 'throttle_gain'))
throttle_offset_link = traitlets.link((throttle_offset, 'value'), (car, 'throttle_offset'))

## 次の作業

[./live_data_collection.ipynb](./live_data_collection.ipynb)を実行し、データセットの収集と学習をおこないます。